In [8]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def prime_factors(n):
    F=factor(n)
    f0=[f[0] for f in F]
    return sortAndSmash(f0)

def omega(n):
    if n==1:return 1
    if n>1:return len(prime_factors(n))

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [9]:
omegaSrs=0
for n in [1..10]:
    omegaSrs=omegaSrs+omega(n)*x^n
print(omegaSrs)
print(omegaSrs.truncate(4))

2*x^10 + x^9 + x^8 + x^7 + 2*x^6 + x^5 + x^4 + x^3 + x^2 + x
x^3 + x^2 + x


In [10]:
import time

omegaSrs=0
start=time.time()
for n in [1..20000]:
    omegaSrs=omegaSrs+omega(n)*x^n
    print((n,time.time()-start))

(1, 0.0049550533294677734)
(2, 0.0054399967193603516)
(3, 0.005522012710571289)
(4, 0.005669116973876953)
(5, 0.005732059478759766)
(6, 0.005851030349731445)
(7, 0.006009101867675781)
(8, 0.006101131439208984)
(9, 0.006141185760498047)
(10, 0.006186962127685547)
(11, 0.0062220096588134766)
(12, 0.006276130676269531)
(13, 0.006850004196166992)
(14, 0.006916999816894531)
(15, 0.006963014602661133)
(16, 0.006999969482421875)
(17, 0.007031917572021484)
(18, 0.007096052169799805)
(19, 0.007133007049560547)
(20, 0.007169008255004883)
(21, 0.007208108901977539)
(22, 0.0073680877685546875)
(23, 0.007472991943359375)
(24, 0.007542848587036133)
(25, 0.007590055465698242)
(26, 0.007637977600097656)
(27, 0.0076751708984375)
(28, 0.007711172103881836)
(29, 0.007745981216430664)
(30, 0.00778508186340332)
(31, 0.007819890975952148)
(32, 0.007856130599975586)
(33, 0.007893085479736328)
(34, 0.007976055145263672)
(35, 0.008041143417358398)
(36, 0.00808405876159668)
(37, 0.008116960525512695)
(38, 0.008

(1566, 0.16652727127075195)
(1567, 0.23259305953979492)
(1568, 0.232666015625)
(1569, 0.2327132225036621)
(1570, 0.23275995254516602)
(1571, 0.23279500007629395)
(1572, 0.23283910751342773)
(1573, 0.2328810691833496)
(1574, 0.2329251766204834)
(1575, 0.23296809196472168)
(1576, 0.23301100730895996)
(1577, 0.23305511474609375)
(1578, 0.23310399055480957)
(1579, 0.23313593864440918)
(1580, 0.2331838607788086)
(1581, 0.2332310676574707)
(1582, 0.23333501815795898)
(1583, 0.23338890075683594)
(1584, 0.23343801498413086)
(1585, 0.23348283767700195)
(1586, 0.23352909088134766)
(1587, 0.23357200622558594)
(1588, 0.23361492156982422)
(1589, 0.2336571216583252)
(1590, 0.2337031364440918)
(1591, 0.23374605178833008)
(1592, 0.23379015922546387)
(1593, 0.23383212089538574)
(1594, 0.23387408256530762)
(1595, 0.23427104949951172)
(1596, 0.234328031539917)
(1597, 0.23436307907104492)
(1598, 0.2344071865081787)
(1599, 0.2344510555267334)
(1600, 0.23449420928955078)
(1601, 0.2345278263092041)
(1602, 0.

(3066, 0.4874899387359619)
(3067, 0.48755407333374023)
(3068, 0.48761582374572754)
(3069, 0.4876749515533447)
(3070, 0.4877328872680664)
(3071, 0.48778796195983887)
(3072, 0.48784804344177246)
(3073, 0.4879109859466553)
(3074, 0.48797106742858887)
(3075, 0.4880521297454834)
(3076, 0.48810791969299316)
(3077, 0.48816514015197754)
(3078, 0.4882221221923828)
(3079, 0.48826003074645996)
(3080, 0.48831915855407715)
(3081, 0.4883759021759033)
(3082, 0.4884331226348877)
(3083, 0.48847293853759766)
(3084, 0.48853111267089844)
(3085, 0.4885878562927246)
(3086, 0.48864293098449707)
(3087, 0.48869800567626953)
(3088, 0.48899006843566895)
(3089, 0.4890320301055908)
(3090, 0.4890890121459961)
(3091, 0.48914504051208496)
(3092, 0.48920321464538574)
(3093, 0.4892580509185791)
(3094, 0.4893150329589844)
(3095, 0.48937010765075684)
(3096, 0.4894261360168457)
(3097, 0.48948121070861816)
(3098, 0.4895358085632324)
(3099, 0.4895920753479004)
(3100, 0.48964810371398926)
(3101, 0.4897031784057617)
(3102, 0.

(4381, 0.6884269714355469)
(4382, 0.688507080078125)
(4383, 0.6885790824890137)
(4384, 0.6886491775512695)
(4385, 0.6887180805206299)
(4386, 0.6887891292572021)
(4387, 0.6888580322265625)
(4388, 0.6889278888702393)
(4389, 0.689000129699707)
(4390, 0.6890709400177002)
(4391, 0.6891181468963623)
(4392, 0.6891889572143555)
(4393, 0.6892571449279785)
(4394, 0.6893632411956787)
(4395, 0.6894371509552002)
(4396, 0.6895101070404053)
(4397, 0.6895589828491211)
(4398, 0.6896300315856934)
(4399, 0.6897001266479492)
(4400, 0.6897718906402588)
(4401, 0.6898410320281982)
(4402, 0.6899139881134033)
(4403, 0.6899869441986084)
(4404, 0.6900570392608643)
(4405, 0.6901359558105469)
(4406, 0.6902120113372803)
(4407, 0.6903131008148193)
(4408, 0.690403938293457)
(4409, 0.6904559135437012)
(4410, 0.6905319690704346)
(4411, 0.6906030178070068)
(4412, 0.6906740665435791)
(4413, 0.6907439231872559)
(4414, 0.6908140182495117)
(4415, 0.6915411949157715)
(4416, 0.6916279792785645)
(4417, 0.6917040348052979)
(441

(5267, 0.8930850028991699)
(5268, 0.8937561511993408)
(5269, 0.8938479423522949)
(5270, 0.8939402103424072)
(5271, 0.8940351009368896)
(5272, 0.8941531181335449)
(5273, 0.8942050933837891)
(5274, 0.8942821025848389)
(5275, 0.8943600654602051)
(5276, 0.8944389820098877)
(5277, 0.8945169448852539)
(5278, 0.8945960998535156)
(5279, 0.8946831226348877)
(5280, 0.8947670459747314)
(5281, 0.894819974899292)
(5282, 0.8949007987976074)
(5283, 0.8949830532073975)
(5284, 0.8950629234313965)
(5285, 0.8951442241668701)
(5286, 0.8952219486236572)
(5287, 0.895301103591919)
(5288, 0.8953812122344971)
(5289, 0.8954629898071289)
(5290, 0.8955419063568115)
(5291, 0.8956210613250732)
(5292, 0.8957011699676514)
(5293, 0.8957810401916504)
(5294, 0.8958618640899658)
(5295, 0.8959410190582275)
(5296, 0.8960192203521729)
(5297, 0.896071195602417)
(5298, 0.8961520195007324)
(5299, 0.8963479995727539)
(5300, 0.8964290618896484)
(5301, 0.896510124206543)
(5302, 0.8965909481048584)
(5303, 0.8966419696807861)
(5304

...

(17456, 3.9957852363586426)
(17457, 3.996467113494873)
(17458, 3.9966719150543213)
(17459, 3.9968690872192383)
(17460, 3.9970619678497314)
(17461, 3.9972660541534424)
(17462, 3.9974610805511475)
(17463, 3.9977529048919678)
(17464, 3.9979610443115234)
(17465, 3.998162031173706)
(17466, 3.9983599185943604)
(17467, 3.998476028442383)
(17468, 3.998667001724243)
(17469, 3.9988651275634766)
(17470, 3.999066114425659)
(17471, 3.999202013015747)
(17472, 3.999406099319458)
(17473, 3.999601125717163)
(17474, 3.9997951984405518)
(17475, 3.9999940395355225)
(17476, 4.000262022018433)
(17477, 4.000385999679565)
(17478, 4.000581979751587)
(17479, 4.000802040100098)
(17480, 4.001007080078125)
(17481, 4.00120210647583)
(17482, 4.001402139663696)
(17483, 4.00151801109314)
(17484, 4.001716136932373)
(17485, 4.001913070678711)
(17486, 4.002104997634888)
(17487, 4.002326965332031)
(17488, 4.002545118331909)
(17489, 4.0026609897613525)
(17490, 4.002867221832275)
(17491, 4.002990007400513)
(17492, 4.0031909

(18057, 4.228626012802124)
(18058, 4.228872060775757)
(18059, 4.229002952575684)
(18060, 4.229205846786499)
(18061, 4.229328155517578)
(18062, 4.229526042938232)
(18063, 4.23001503944397)
(18064, 4.230218887329102)
(18065, 4.230419158935547)
(18066, 4.230631113052368)
(18067, 4.230839014053345)
(18068, 4.231045961380005)
(18069, 4.23152494430542)
(18070, 4.231734991073608)
(18071, 4.231935262680054)
(18072, 4.232141971588135)
(18073, 4.232343912124634)
(18074, 4.2325520515441895)
(18075, 4.233138084411621)
(18076, 4.233474969863892)
(18077, 4.233644962310791)
(18078, 4.233978033065796)
(18079, 4.234325885772705)
(18080, 4.235189914703369)
(18081, 4.235467910766602)
(18082, 4.235719203948975)
(18083, 4.236043930053711)
(18084, 4.236946105957031)
(18085, 4.237294912338257)
(18086, 4.2376391887664795)
(18087, 4.238302230834961)
(18088, 4.238516807556152)
(18089, 4.238644123077393)
(18090, 4.238847970962524)
(18091, 4.239054918289185)
(18092, 4.239262104034424)
(18093, 4.239751100540161)
(

(18707, 4.4287519454956055)
(18708, 4.429605960845947)
(18709, 4.429876804351807)
(18710, 4.4301841259002686)
(18711, 4.430393934249878)
(18712, 4.430677175521851)
(18713, 4.430827856063843)
(18714, 4.431042909622192)
(18715, 4.431246042251587)
(18716, 4.4314470291137695)
(18717, 4.431658983230591)
(18718, 4.431865930557251)
(18719, 4.43198823928833)
(18720, 4.432329177856445)
(18721, 4.432538986206055)
(18722, 4.432749032974243)
(18723, 4.432955026626587)
(18724, 4.4331629276275635)
(18725, 4.433364152908325)
(18726, 4.433567047119141)
(18727, 4.433861017227173)
(18728, 4.434118986129761)
(18729, 4.43438196182251)
(18730, 4.4346699714660645)
(18731, 4.434891939163208)
(18732, 4.435135841369629)
(18733, 4.436151027679443)
(18734, 4.436399936676025)
(18735, 4.436625003814697)
(18736, 4.436832904815674)
(18737, 4.43815016746521)
(18738, 4.438554048538208)
(18739, 4.439445972442627)
(18740, 4.439850091934204)
(18741, 4.440293073654175)
(18742, 4.440783977508545)
(18743, 4.440927267074585)

(19311, 4.637799024581909)
(19312, 4.638672113418579)
(19313, 4.638895034790039)
(19314, 4.639112234115601)
(19315, 4.639331102371216)
(19316, 4.639580011367798)
(19317, 4.6397950649261475)
(19318, 4.640010118484497)
(19319, 4.640130043029785)
(19320, 4.640348196029663)
(19321, 4.640474081039429)
(19322, 4.640697002410889)
(19323, 4.6409220695495605)
(19324, 4.641155004501343)
(19325, 4.641373157501221)
(19326, 4.641592979431152)
(19327, 4.641813278198242)
(19328, 4.642030954360962)
(19329, 4.642250061035156)
(19330, 4.642506837844849)
(19331, 4.642956972122192)
(19332, 4.643234968185425)
(19333, 4.64341402053833)
(19334, 4.643647909164429)
(19335, 4.643872022628784)
(19336, 4.64408802986145)
(19337, 4.64435601234436)
(19338, 4.644589900970459)
(19339, 4.644813060760498)
(19340, 4.645040988922119)
(19341, 4.645258903503418)
(19342, 4.645539999008179)
(19343, 4.645796060562134)
(19344, 4.646585941314697)
(19345, 4.64683198928833)
(19346, 4.647058010101318)
(19347, 4.647278070449829)
(19

(19957, 4.844948053359985)
(19958, 4.845484972000122)
(19959, 4.845715045928955)
(19960, 4.84593391418457)
(19961, 4.846061944961548)
(19962, 4.846281051635742)
(19963, 4.846414089202881)
(19964, 4.846638917922974)
(19965, 4.846896171569824)
(19966, 4.8471221923828125)
(19967, 4.8473451137542725)
(19968, 4.847774028778076)
(19969, 4.848081111907959)
(19970, 4.848323106765747)
(19971, 4.84873104095459)
(19972, 4.849009990692139)
(19973, 4.849139213562012)
(19974, 4.849353075027466)
(19975, 4.84962797164917)
(19976, 4.849914073944092)
(19977, 4.850293874740601)
(19978, 4.850576162338257)
(19979, 4.850710868835449)
(19980, 4.850934982299805)
(19981, 4.8512489795684814)
(19982, 4.851500034332275)
(19983, 4.852776050567627)
(19984, 4.853102922439575)
(19985, 4.853986978530884)
(19986, 4.8542492389678955)
(19987, 4.8545849323272705)
(19988, 4.854823112487793)
(19989, 4.85559606552124)
(19990, 4.855823993682861)
(19991, 4.85598611831665)
(19992, 4.856216907501221)
(19993, 4.856348037719727)
(

In [11]:
import time

def recip_omega(n):
    g=L(1/omegaSrs.truncate(2*n))
    return g

def recip_omegaPowerConstant(n):
    f=recip_omega(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=10000)
import pickle
datafile= open('/Users/barrybrent/run14may23no2.txt','wb') 
data=[]
start=time.time()/3600
for a in [1..2000]:
    c=recip_omegaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1, 3.033783286809921e-07)
(2, 8.97503923624754e-07)
(3, 2.1669548004865646e-06)
(4, 0.00014279806055128574)
(5, 0.00035774719435721636)
(6, 0.0005454372148960829)
(7, 0.0007924771634861827)
(8, 0.0009839002741500735)
(9, 0.0012213971931487322)
(10, 0.001458811922930181)
(11, 0.0017706997459754348)
(12, 0.002021611377131194)
(13, 0.0023191144573502243)
(14, 0.002615754143334925)
(15, 0.002985299506690353)
(16, 0.0032113888300955296)
(17, 0.0035097638610750437)
(18, 0.0038192483480088413)
(19, 0.004170326981693506)
(20, 0.004468383325729519)
(21, 0.004834845254663378)
(22, 0.005188112205360085)
(23, 0.005612610839307308)
(24, 0.005903861136175692)
(25, 0.006254700885619968)
(26, 0.006615398917347193)
(27, 0.00703300500754267)
(28, 0.007396344735752791)
(29, 0.007819981896318495)
(30, 0.008237226051278412)
(31, 0.008718309458345175)
(32, 0.009003193292301148)
(33, 0.009357146103866398)
(34, 0.0097120082937181)
(35, 0.01013624801998958)
(36, 0.010489046340808272)
(37, 0.01090391306206584)

(308, 0.1732355285785161)
(309, 0.17404223833000287)
(310, 0.17484519025310874)
(311, 0.17570489994250238)
(312, 0.17646383721148595)
(313, 0.17727321665734053)
(314, 0.17807673354400322)
(315, 0.1789424875169061)
(316, 0.17974844633135945)
(317, 0.1806198353297077)
(318, 0.18148557608947158)
(319, 0.18242019467288628)
(320, 0.18303043581545353)
(321, 0.18370669498108327)
(322, 0.1843858283245936)
(323, 0.18513230857206509)
(324, 0.18580757244490087)
(325, 0.18655509391101077)
(326, 0.18729870219249278)
(327, 0.18811276805354282)
(328, 0.18879978999029845)
(329, 0.1895459988154471)
(330, 0.1902902030851692)
(331, 0.1910949208540842)
(332, 0.19183255278039724)
(333, 0.1926411088788882)
(334, 0.19346024334663525)
(335, 0.1943306746543385)
(336, 0.1950059644295834)
(337, 0.1957500580465421)
(338, 0.1964993632864207)
(339, 0.19730195525335148)
(340, 0.19803949916968122)
(341, 0.19884523388464004)
(342, 0.19965303753269836)
(343, 0.20052262168610469)
(344, 0.20126521110069007)
(345, 0.20208

(618, 0.43434746330603957)
(619, 0.4353304008836858)
(620, 0.4362516024848446)
(621, 0.43723171134479344)
(622, 0.4382157194777392)
(623, 0.43925506056984887)
(624, 0.4401042718673125)
(625, 0.4410323324846104)
(626, 0.44195979833602905)
(627, 0.44293741637375206)
(628, 0.4438444750267081)
(629, 0.44482211949070916)
(630, 0.4457959913997911)
(631, 0.44683592551155016)
(632, 0.4477524916292168)
(633, 0.4487238291767426)
(634, 0.4497010125196539)
(635, 0.45075884327525273)
(636, 0.45174845558358356)
(637, 0.452782302221749)
(638, 0.45381697587436065)
(639, 0.454917689145077)
(640, 0.4556318038376048)
(641, 0.456420885224361)
(642, 0.45720469194930047)
(643, 0.45805486134486273)
(644, 0.45884324749931693)
(645, 0.459706234105397)
(646, 0.4605576427420601)
(647, 0.46146731806220487)
(648, 0.4622492802445777)
(649, 0.4631020605447702)
(650, 0.46394986112136394)
(651, 0.46485993382520974)
(652, 0.4657054764102213)
(653, 0.4666182681103237)
(654, 0.4675259927753359)
(655, 0.46850760700181127)

(934, 0.7449262277805246)
(935, 0.7460479725268669)
(936, 0.7470378730795346)
(937, 0.7480967741576023)
(938, 0.749149099458009)
(939, 0.7502703702193685)
(940, 0.7513215513317846)
(941, 0.7524578068987466)
(942, 0.7536103791790083)
(943, 0.7548219066811725)
(944, 0.7558204235974699)
(945, 0.7568891005939804)
(946, 0.7579544374602847)
(947, 0.7590721113956533)
(948, 0.760140253813006)
(949, 0.761266682529822)
(950, 0.7623879005550407)
(951, 0.7635673355543986)
(952, 0.7646272982819937)
(953, 0.7657458685571328)
(954, 0.7668680680217221)
(955, 0.7680515355314128)
(956, 0.7691711299703456)
(957, 0.7703598132939078)
(958, 0.7715632427716628)
(959, 0.7728078289073892)
(960, 0.7737439838820137)
(961, 0.7747360361390747)
(962, 0.7757269311114214)
(963, 0.7767885866342112)
(964, 0.777782158635091)
(965, 0.7788496486027725)
(966, 0.7799277225276455)
(967, 0.7810776261030696)
(968, 0.7820752582629211)
(969, 0.7832169105531648)
(970, 0.784320566104725)
(971, 0.7854623480816372)
(972, 0.786526945

(1242, 1.0684289552737027)
(1243, 1.069589528895449)
(1244, 1.0706874841125682)
(1245, 1.07186747668311)
(1246, 1.0730329463840462)
(1247, 1.0742601900128648)
(1248, 1.0752398801851086)
(1249, 1.0762822274700738)
(1250, 1.0773526380653493)
(1251, 1.0784471460501663)
(1252, 1.0794806013582274)
(1253, 1.080578989116475)
(1254, 1.0816789211239666)
(1255, 1.0828381280880421)
(1256, 1.0838679450098425)
(1257, 1.0849784094607458)
(1258, 1.0860796091728844)
(1259, 1.0872361458605155)
(1260, 1.0883371252566576)
(1261, 1.0895002271863632)
(1262, 1.09065999527229)
(1263, 1.091884252789896)
(1264, 1.092929282458499)
(1265, 1.0940328011056408)
(1266, 1.0951419033808634)
(1267, 1.096305828599725)
(1268, 1.0974067358765751)
(1269, 1.0985743841156363)
(1270, 1.0997379202744924)
(1271, 1.100967118050903)
(1272, 1.1020726594724692)
(1273, 1.1032422482967377)
(1274, 1.1044310693978332)
(1275, 1.105659026070498)
(1276, 1.1068251724937)
(1277, 1.108049206144642)
(1278, 1.1092771386611275)
(1279, 1.1105790

(1549, 1.4282771905418485)
(1550, 1.4294143986771815)
(1551, 1.4306104857823811)
(1552, 1.4316087688785046)
(1553, 1.4326871108496562)
(1554, 1.4337637613643892)
(1555, 1.4349195119575597)
(1556, 1.4360506774974056)
(1557, 1.4372098374878988)
(1558, 1.4383581805159338)
(1559, 1.4395493614138104)
(1560, 1.4406537591130473)
(1561, 1.441798979358282)
(1562, 1.4429531199857593)
(1563, 1.444147858012002)
(1564, 1.445274643599987)
(1565, 1.4464714949717745)
(1566, 1.4476556536392309)
(1567, 1.448912020539865)
(1568, 1.4499118702951819)
(1569, 1.450966224714648)
(1570, 1.4520349614322186)
(1571, 1.4531686302507296)
(1572, 1.454235459677875)
(1573, 1.455367537215352)
(1574, 1.4565008513745852)
(1575, 1.4576965394080617)
(1576, 1.4587556636543013)
(1577, 1.4598919207928702)
(1578, 1.461009188380558)
(1579, 1.4622026241268031)
(1580, 1.4633317075204104)
(1581, 1.46451598132262)
(1582, 1.4656955846585333)
(1583, 1.466962781676557)
(1584, 1.4680227591306902)
(1585, 1.469146070536226)
(1586, 1.4702

(1856, 1.805292381613981)
(1857, 1.8065070497104898)
(1858, 1.8077130416058935)
(1859, 1.809001240239013)
(1860, 1.810234478849452)
(1861, 1.8115185735514387)
(1862, 1.812808316317387)
(1863, 1.8141514922026545)
(1864, 1.8153710705810227)
(1865, 1.816653554677032)
(1866, 1.8179360999492928)
(1867, 1.8192887300392613)
(1868, 1.8205707072047517)
(1869, 1.8219231194234453)
(1870, 1.8232847199542448)
(1871, 1.8247102707973681)
(1872, 1.8259626539074816)
(1873, 1.8272960349568166)
(1874, 1.8286066896980628)
(1875, 1.8299748044228181)
(1876, 1.8312742908019572)
(1877, 1.832670656440314)
(1878, 1.8340349530335516)
(1879, 1.8354653358110227)
(1880, 1.8367483794572763)
(1881, 1.8381060342071578)
(1882, 1.8394576066639274)
(1883, 1.8408689458156005)
(1884, 1.8422245946712792)
(1885, 1.8436452664318494)
(1886, 1.8450620521907695)
(1887, 1.8465310413739644)
(1888, 1.847746038052719)
(1889, 1.8490417085704394)
(1890, 1.8503195886150934)
(1891, 1.851673324184958)
(1892, 1.8529544444172643)
(1893, 1.

In [12]:
import time

def recip_omega(n):
    g=L(1/omegaSrs.truncate(2*n))
    return g

def recip_omegaPowerConstant(n):
    f=recip_omega(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run14may23no3.txt','wb') 
data=[]
start=time.time()/3600
for a in [2001..5000]:
    c=recip_omegaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(2001, 0.001743017986882478)
(2002, 0.0034775891108438373)
(2003, 0.005335655820090324)
(2004, 0.0070779319503344595)
(2005, 0.008895994920749217)
(2006, 0.010711088019888848)
(2007, 0.012566336896270514)
(2008, 0.014304370270110667)
(2009, 0.016103832749649882)
(2010, 0.017904505832120776)
(2011, 0.019763495540246367)
(2012, 0.0215595118352212)
(2013, 0.02344606746919453)
(2014, 0.02530762186506763)
(2015, 0.0272339676739648)
(2016, 0.028898588614538312)
(2017, 0.030634247174020857)
(2018, 0.032353952701669186)
(2019, 0.03414289635838941)
(2020, 0.03588980186032131)
(2021, 0.0376891263294965)
(2022, 0.039520291320513934)
(2023, 0.04140310385264456)
(2024, 0.04313282668590546)
(2025, 0.044933549710549414)
(2026, 0.046746273001190275)
(2027, 0.0486249549430795)
(2028, 0.0504801538772881)
(2029, 0.05241636140272021)
(2030, 0.05427765048807487)
(2031, 0.0562353107961826)
(2032, 0.057978838798590004)
(2033, 0.05989387776935473)
(2034, 0.06183588464045897)
(2035, 0.06381208600942045)
(2036,

(2298, 0.45946146082133055)
(2299, 0.461269628838636)
(2300, 0.46298910572659224)
(2301, 0.46478853770531714)
(2302, 0.4666021938319318)
(2303, 0.4684714382747188)
(2304, 0.46969151188386604)
(2305, 0.47108332108473405)
(2306, 0.47247872583102435)
(2307, 0.4739236104651354)
(2308, 0.4753010480199009)
(2309, 0.47674418054521084)
(2310, 0.4782073990791105)
(2311, 0.47972157993353903)
(2312, 0.4811015132581815)
(2313, 0.48255014774622396)
(2314, 0.48400690336711705)
(2315, 0.4855136690894142)
(2316, 0.4869606966385618)
(2317, 0.4884789193747565)
(2318, 0.4899937512818724)
(2319, 0.4915663338615559)
(2320, 0.4929414643556811)
(2321, 0.49438164418097585)
(2322, 0.4958215854712762)
(2323, 0.4973288038163446)
(2324, 0.49876581999706104)
(2325, 0.5002788846613839)
(2326, 0.5017819677013904)
(2327, 0.5033684890950099)
(2328, 0.5048229796811938)
(2329, 0.506337814964354)
(2330, 0.5078438321361318)
(2331, 0.5094315508031286)
(2332, 0.5109447427093983)
(2333, 0.5125410024775192)
(2334, 0.514134427

(2602, 0.9537702130619437)
(2603, 0.9553649661247618)
(2604, 0.9568994799046777)
(2605, 0.9585027283173986)
(2606, 0.9600917766219936)
(2607, 0.96175527269952)
(2608, 0.9632121502654627)
(2609, 0.9647262280341238)
(2610, 0.9662451741169207)
(2611, 0.967821929953061)
(2612, 0.9693383508129045)
(2613, 0.9709301477414556)
(2614, 0.9725159890949726)
(2615, 0.9741662560845725)
(2616, 0.9756811077240855)
(2617, 0.9772652994142845)
(2618, 0.9788957682903856)
(2619, 0.9805795546853915)
(2620, 0.9822516952408478)
(2621, 0.9839175550150685)
(2622, 0.9855941304704174)
(2623, 0.9873081689001992)
(2624, 0.9886874174699187)
(2625, 0.9901327877305448)
(2626, 0.9915947319241241)
(2627, 0.9931087468285114)
(2628, 0.9945441619493067)
(2629, 0.9960514602717012)
(2630, 0.9975646618986502)
(2631, 0.9991451410460286)
(2632, 1.000586665526498)
(2633, 1.0020973591599613)
(2634, 1.0036247010575607)
(2635, 1.0052274785703048)
(2636, 1.0067771638277918)
(2637, 1.0083588219131343)
(2638, 1.0099413102143444)
(2639

(2909, 1.4741724655032158)
(2910, 1.475970146653708)
(2911, 1.4778446588898078)
(2912, 1.4794777782517485)
(2913, 1.4811771818785928)
(2914, 1.4828479799907655)
(2915, 1.4845956215867773)
(2916, 1.4862781765987165)
(2917, 1.4880304333055392)
(2918, 1.48977342719445)
(2919, 1.4915882705245167)
(2920, 1.4932566119241528)
(2921, 1.4949993690825067)
(2922, 1.4967331177322194)
(2923, 1.4985403735772707)
(2924, 1.5002951844362542)
(2925, 1.5021073188399896)
(2926, 1.5039050418999977)
(2927, 1.505794421886094)
(2928, 1.5074607764254324)
(2929, 1.5092809744528495)
(2930, 1.5111086318502203)
(2931, 1.5130008977139369)
(2932, 1.5148414799477905)
(2933, 1.5167372415889986)
(2934, 1.5186279391054995)
(2935, 1.520571635745)
(2936, 1.5224016801803373)
(2937, 1.5242859412683174)
(2938, 1.5261706821620464)
(2939, 1.5281252930290066)
(2940, 1.5300253535970114)
(2941, 1.5319880691240542)
(2942, 1.5339685552171431)
(2943, 1.535991049662698)
(2944, 1.5375145705183968)
(2945, 1.5391483466955833)
(2946, 1.5

(3216, 2.0043075474677607)
(3217, 2.006056083308067)
(3218, 2.007683656120207)
(3219, 2.009358430223074)
(3220, 2.0109768577385694)
(3221, 2.0126450422103517)
(3222, 2.0143293894361705)
(3223, 2.016071845020633)
(3224, 2.0176778452587314)
(3225, 2.019343843567185)
(3226, 2.021009589661844)
(3227, 2.022759459738154)
(3228, 2.024426966905594)
(3229, 2.0261530090938322)
(3230, 2.0278932877699845)
(3231, 2.0296909452299587)
(3232, 2.0312249968992546)
(3233, 2.032833492208738)
(3234, 2.034443951328285)
(3235, 2.036116977164056)
(3236, 2.0377200844231993)
(3237, 2.0393890774575993)
(3238, 2.041059542738367)
(3239, 2.042792886844836)
(3240, 2.0443954811198637)
(3241, 2.0460600654478185)
(3242, 2.047729770769365)
(3243, 2.0494773904792964)
(3244, 2.0511503375018947)
(3245, 2.0528824866050854)
(3246, 2.054633022460621)
(3247, 2.0564340830314904)
(3248, 2.058039329946041)
(3249, 2.059710707515478)
(3250, 2.061383892199956)
(3251, 2.063128008041531)
(3252, 2.0648057788494043)
(3253, 2.06654838193

(3526, 2.5579755107755773)
(3527, 2.559979782497976)
(3528, 2.561849548306782)
(3529, 2.5637958682491444)
(3530, 2.56572849053191)
(3531, 2.567740082158707)
(3532, 2.5696734968805686)
(3533, 2.571804389124736)
(3534, 2.5738256471813656)
(3535, 2.5759102460579015)
(3536, 2.5777899243985303)
(3537, 2.5797291030175984)
(3538, 2.581673110544216)
(3539, 2.5836867022444494)
(3540, 2.5856190841295756)
(3541, 2.587627726374194)
(3542, 2.5896373199648224)
(3543, 2.5917088472051546)
(3544, 2.593661800783593)
(3545, 2.595668914669659)
(3546, 2.5976904805283993)
(3547, 2.5997665188624524)
(3548, 2.601767493353691)
(3549, 2.6038376194192097)
(3550, 2.60592331411317)
(3551, 2.6080639702267945)
(3552, 2.609930833801627)
(3553, 2.611898215545807)
(3554, 2.6138544435962103)
(3555, 2.6158759885584004)
(3556, 2.6178430391591974)
(3557, 2.61986488715047)
(3558, 2.6218882396933623)
(3559, 2.623971212771721)
(3560, 2.6259170958655886)
(3561, 2.6279285246273503)
(3562, 2.629968449473381)
(3563, 2.63207074638

(3836, 3.157744387513958)
(3837, 3.159971766290255)
(3838, 3.162201124650892)
(3839, 3.164509379712399)
(3840, 3.1661734204972163)
(3841, 3.1679921913309954)
(3842, 3.1698120393557474)
(3843, 3.17169121443294)
(3844, 3.173481134173926)
(3845, 3.1753663293784484)
(3846, 3.177241951052565)
(3847, 3.1791790802381)
(3848, 3.181001477234531)
(3849, 3.182881092710886)
(3850, 3.1847656901809387)
(3851, 3.186718981654849)
(3852, 3.1886076933005825)
(3853, 3.190555709414184)
(3854, 3.1924930082750507)
(3855, 3.194500207726378)
(3856, 3.1963001949479803)
(3857, 3.1981671616085805)
(3858, 3.2000500935828313)
(3859, 3.201987272768747)
(3860, 3.2038755527464673)
(3861, 3.2058401071699336)
(3862, 3.207792103057727)
(3863, 3.209818644099869)
(3864, 3.211689861898776)
(3865, 3.2136429502279498)
(3866, 3.2156030640471727)
(3867, 3.217616111622192)
(3868, 3.219558375538327)
(3869, 3.221587957756128)
(3870, 3.223604401340708)
(3871, 3.225690357736312)
(3872, 3.227493141312152)
(3873, 3.2293710632948205)


(4146, 3.777957156067714)
(4147, 3.779616276093293)
(4148, 3.7811946199508384)
(4149, 3.7828460175078362)
(4150, 3.784521196037531)
(4151, 3.786238029191736)
(4152, 3.787863988080062)
(4153, 3.7895154224243015)
(4154, 3.7911628172150813)
(4155, 3.7928738823975436)
(4156, 3.794518096372485)
(4157, 3.796249004662968)
(4158, 3.797966681071557)
(4159, 3.7997462771600112)
(4160, 3.8011833532946184)
(4161, 3.8027095313882455)
(4162, 3.8042317632352933)
(4163, 3.805817026353907)
(4164, 3.807336397410836)
(4165, 3.8089214346255176)
(4166, 3.8105032147141173)
(4167, 3.8121504599112086)
(4168, 3.8136733502615243)
(4169, 3.815254442160949)
(4170, 3.8168462638859637)
(4171, 3.8184994410257787)
(4172, 3.820077783893794)
(4173, 3.821721105312463)
(4174, 3.8233671680209227)
(4175, 3.8250841241679154)
(4176, 3.826592983561568)
(4177, 3.8281622551730834)
(4178, 3.8297448030789383)
(4179, 3.8313903733505867)
(4180, 3.8329748408286832)
(4181, 3.8346198400249705)
(4182, 3.8362596163060516)
(4183, 3.837980

(4459, 4.339546040806454)
(4460, 4.341485458833631)
(4461, 4.3434838155517355)
(4462, 4.345472144370433)
(4463, 4.34753742441535)
(4464, 4.349394513876177)
(4465, 4.35139372444246)
(4466, 4.353396246035118)
(4467, 4.355475216638297)
(4468, 4.357479508849792)
(4469, 4.359549468266778)
(4470, 4.361622252152301)
(4471, 4.363758369407151)
(4472, 4.365766582195647)
(4473, 4.367854495765641)
(4474, 4.369935157184955)
(4475, 4.37208865780849)
(4476, 4.3741606313269585)
(4477, 4.376310986874159)
(4478, 4.378446755581535)
(4479, 4.380656004708726)
(4480, 4.382364128250629)
(4481, 4.384158126602415)
(4482, 4.385947232658509)
(4483, 4.38780920044519)
(4484, 4.389685973059386)
(4485, 4.391620495240204)
(4486, 4.393579899624456)
(4487, 4.3956128197023645)
(4488, 4.397505417466164)
(4489, 4.399502033542376)
(4490, 4.401463087997399)
(4491, 4.403497112449259)
(4492, 4.405458536406513)
(4493, 4.407494253886398)
(4494, 4.409526679082774)
(4495, 4.411633468582295)
(4496, 4.413527256634552)
(4497, 4.4154

(4774, 4.966453777451534)
(4775, 4.968546361080371)
(4776, 4.970502045529429)
(4777, 4.972519711882342)
(4778, 4.974565653828904)
(4779, 4.976640933833551)
(4780, 4.978661576111335)
(4781, 4.980754275282379)
(4782, 4.982842290250119)
(4783, 4.9849867091397755)
(4784, 4.986940558592323)
(4785, 4.988959561334923)
(4786, 4.990973479405511)
(4787, 4.993053851125296)
(4788, 4.99506289238343)
(4789, 4.997150518815033)
(4790, 4.999219901044853)
(4791, 5.001350440841634)
(4792, 5.00336765270913)
(4793, 5.005454053520225)
(4794, 5.007535830256529)
(4795, 5.009669798600953)
(4796, 5.011743603856303)
(4797, 5.0139025778044015)
(4798, 5.016045328811742)
(4799, 5.01825240382459)
(4800, 5.02012855745852)
(4801, 5.022075132001191)
(4802, 5.024025181890465)
(4803, 5.026034099981189)
(4804, 5.027986906701699)
(4805, 5.030005976383109)
(4806, 5.03201900300337)
(4807, 5.034099259937648)
(4808, 5.0360406211111695)
(4809, 5.038050668605138)
(4810, 5.040053419885226)
(4811, 5.042141503246967)
(4812, 5.04415